In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']#['svg']
import matplotlib.pyplot as plt
# from qutip import *
import numpy as np

from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d, spinless_fermion_basis_1d, basis_1d # Hilbert space spin basis

# spin bugs

In [24]:
# spin bugs
L=3
basis_spin = spin_basis_1d(L=L)
basis_spin_zblock = spin_basis_1d(L=L, zblock=1)
dynamic_spin = []
static_spin_aa = [[["+-",[[1.0,0,2]]]]]
static_spin_aa += [[["-+",[[1.0,0,2]]]]]

aa_spin = [hamiltonian(aa_op,dynamic_spin,basis=basis_spin,
                        check_pcon=False,check_symm=False,check_herm=False)
           for aa_op in static_spin_aa]
aa_spin_zblock = [hamiltonian(aa_op,dynamic_spin,basis=basis_spin_zblock,
                        check_pcon=False,check_symm=False,check_herm=False)
           for aa_op in static_spin_aa]

Without symmetry "+-" is transposed of "-+"

In [25]:
aa_spin[1].static - aa_spin[0].static.T

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>

With symmetry "+-" is brocken

In [26]:
aa_spin_zblock[1].static - aa_spin_zblock[0].static.T

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

It is actually zero:

In [27]:
aa_spin_zblock[0].static

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>

Additionally "+" and "-" are defined with extra factor of 2:

In [28]:
aa_spin[0].static.toarray().real

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 4., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 4., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

# Fermion bug

In [94]:
##### define model parameters #####

L=3 # system size
J=1.0 # spin zz interaction


# zblock,PBC = (-1,1)
zblock,PBC = (1,-1)

ener_list = []

##### define fermion model
# define site-coupling lists for external field
h_pot=[[2.0,i] for i in range(L)]

if PBC==1: # periodic BC: odd particle number subspace only
    J_pm=[[-J,i,(i+1)%L] for i in range(L)] # PBC
    J_mp=[[+J,i,(i+1)%L] for i in range(L)] # PBC
    J_pp=[[-J,i,(i+1)%L] for i in range(L)] # PBC
    J_mm=[[+J,i,(i+1)%L] for i in range(L)] # PBC
    # construct fermion basis in the odd particle number subsector
    basis_fermion = spinless_fermion_basis_1d(L=L)#,Nf=range(1,L+1,2))
elif PBC==-1: # anti-periodic BC: even particle number subspace only
    # define bulk site coupling lists
    J_pm=[[-J,i,i+1] for i in range(L-1)]
    J_mp=[[+J,i,i+1] for i in range(L-1)]
    J_pp=[[-J,i,i+1] for i in range(L-1)]
    J_mm=[[+J,i,i+1] for i in range(L-1)]
    # add boundary coupling between sites (L-1,0)
    J_pm.append([+J,L-1,0]) # APBC
    J_mp.append([-J,L-1,0]) # APBC
    J_pp.append([+J,L-1,0]) # APBC
    J_mm.append([-J,L-1,0]) # APBC
     # construct fermion basis in the even particle number subsector
    basis_fermion = spinless_fermion_basis_1d(L = L)#, kblock=0)#, Nf=range(0,L+1,2), kblock=0)
    basis_fermion_kblock = spinless_fermion_basis_1d(L = L, kblock=0)

 # define fermionic static and dynamic lists
static_fermion =[["+-",J_pm],["-+",J_mp],["++",J_pp],["--",J_mm]]
static_fermion_x = [["z",h_pot]]

static_fermion_aa = [[["+-",[[1.0,0,2]]]]]
static_fermion_aa += [[["+-",[[1.0,2,0]]]]]
static_fermion_aa += [[["++",[[1.0,0,2]]]]]
static_fermion_aa += [[["--",[[1.0,2,0]]]]]

dynamic_fermion=[]
# build fermionic Hamiltonian
H_fermion_zz = hamiltonian(static_fermion,dynamic_fermion,basis=basis_fermion,
                        check_pcon=False,check_symm=False, dtype=np.float64)
H_fermion_zz_kblock = hamiltonian(static_fermion,dynamic_fermion,basis=basis_fermion_kblock,
                        check_pcon=False,check_symm=False, dtype=np.float64)

aaaa_ferm = hamiltonian(static_fermion_aaaa,dynamic_fermion,basis=basis_fermion,
                        check_pcon=False,check_symm=False,check_herm=False)
aaaa_ferm_kblock = hamiltonian(static_fermion_aaaa,dynamic_fermion,basis=basis_fermion_kblock,
                        check_pcon=False,check_symm=False,check_herm=False)
aa_ferm = [hamiltonian(a_op,dynamic_fermion,basis=basis_fermion,
                        check_pcon=False,check_symm=False,check_herm=False) 
           for a_op in static_fermion_aa]
aa_ferm_kblock = [hamiltonian(a_op,dynamic_fermion,basis=basis_fermion_kblock,
                        check_pcon=False,check_symm=False,check_herm=False) 
           for a_op in static_fermion_aa]

Hermiticity check passed!
Hermiticity check passed!


In [95]:
H_fermion_zz.static - H_fermion_zz.static.T

<8x8 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

kblock breaks hamiltonian

In [96]:
H_fermion_zz_kblock.static - H_fermion_zz_kblock.static.T

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [97]:
H_fermion_zz_kblock.static.toarray()

array([[0.        , 0.        , 1.73205081, 0.        ],
       [0.        , 0.        , 0.        , 0.57735027],
       [0.57735027, 0.        , 0.        , 0.        ],
       [0.        , 1.73205081, 0.        , 0.        ]])

Correlator without symmetry works

In [98]:
aa_ferm[0].static - aa_ferm[1].static.T

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>

In [99]:
aa_ferm[2].static - aa_ferm[3].static.T

<8x8 sparse matrix of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>

With kblock it is broken

In [100]:
aa_ferm_kblock[0].static - aa_ferm_kblock[1].static.T

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>

In [101]:
aa_ferm_kblock[2].static - aa_ferm_kblock[3].static.T

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Row format>